In [1]:
using Plots, StatsBase, TwoDimensional,OptimPackNextGen
include("../src/SphereIFSCalib.jl")

self

In [16]:
# wavelengths
λ1 = 987.72e-9# laser 1
λ2 = 1123.71e-9# laser 2
λ3 = 1309.37e-9# laser 3
λ4 = 1545.10e-9  # laser 4
λlaser = [λ1,λ2,λ3,λ4]
λ0 = mean(λlaser)# reference

# model of the LensletModel
bbox = BoundingBox(xmin=1, ymin=1, xmax=10, ymax=50);
#print(typeof(bbox))

data = rand(100,100)
print("data: ", typeof(data))
weight = Float64.(rand(100,100).>0.1);


Cinit = rand(Float64,2,4);
print("Cinit: ",  typeof(Cinit))
a0= rand(Float64,4);
print("a0: ",  typeof(a0))
fwhm0= rand(Float64,4)*10;
print("fwhm0: ",  typeof(fwhm0))
laser =  LaserModel(λlaser,a0,fwhm0)
lmod = LensletModel(λ0,laser.nλ-1,bbox);



Cinit = [[ 6.2 10 0 0 ]; [20 100 0 0]]
UpdateDispModel(lmod.dmodel,Cinit)
limage = LensletLaserImage(lmod,laser)
heatmap(limage)

ldata = view(data, lmod.bbox);
lkl = LikelihoodIFS(lmod,laser, ldata);
cost(x::Vector{Float64}) = lkl(x)::Float64
xinit = vcat([a0[:],fwhm0[:],Cinit[:]]...);
print("xinit: ",  typeof(xinit))
xopt = vmlmb(cost, xinit; verb=50);
print("xopt: ",  typeof(xopt))

data: Array{Float64,2}Cinit: Array{Float64,2}a0: Array{Float64,1}fwhm0: Array{Float64,1}xinit: Array{Float64,1}# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    1.4135860247650928E+02  2.28E+01  0.00E+00
    50     61      0    4.3147433009663914E+01  2.34E-01  1.00E+00
   100    124      0    4.3127160589753395E+01  4.84E-01  1.00E+00
   150    187      0    4.3123728882731292E+01  1.24E-01  2.78E-01
xopt: Array{Float64,1}